**Exercise 2.1.** You can verify that the analytical solution to the integral of the function
\begin{equation*}
  g(x)=0.1x^4 -1.5x^3 + 0.53x^2 + 2x + 1
\end{equation*}
between $x=-10$ and $x=10$ is $\int_{-10}^{10} g(x)dx = 4,373.3\bar{3}$. Write a Python function that will take as arguments an anonymous function that the user specifies representing $g(x)$, integration bounds $a$ and $b$, the number of intervals $N$, and
```python
method = {'midpoint', 'trapezoid', 'Simpsons'}
```
Using the composite methods, evaluate the numerical approximations of the integral $\int_a^b g(x)dx$ using all three Newton-Cotes methods in your function and compare the difference between the values of these integrals to the true analytical value of the integral.

In [1]:
import numpy as np
def g_x_1(x):
    g = 0.1 * (x ** 4) - 1.5 * (x ** 3) + 0.53 * (x ** 2) + 2 * x + 1
    return g

def integr_g_x_1(g_x_1, a, b, N, method = 'midpoint'):
    # calculate vector of N + 1 bar bounds
    bin_cuts = np.linspace(a, b, N + 1)
    
    if method == 'midpoint':
        # calculate vector of midpoints
        mids = (bin_cuts[1:] + bin_cuts[:-1]) / 2
        
        # evaluate function at the midpoint
        width = (b - a) / N
        heights = sum(g_x_1(mids))
    
    elif method == 'trapezoid':
        width = (b - a) / (2 * N)
        heights = g_x_1(bin_cuts[0]) + g_x_1(bin_cuts[-1]) + 2 * sum(g_x_1(bin_cuts[1:-1]))
    
    elif method == 'Simpsons':
        bin_cuts = np.linspace(a, b, 2 * N + 1)
        width = (b - a) / (6 * N)
        heights = g_x_1(bin_cuts[0]) + 4 * sum(g_x_1(bin_cuts[1:-1][::2])) + 2 * sum(g_x_1(bin_cuts[1:-1][1::2])) + g_x_1(bin_cuts[-1])
            
    # Add up the area of all the bars
    approx_integr = heights * width
    
    return approx_integr

In [8]:
mid = integr_g_x_1(g_x_1, -10, 10, 200, 'midpoint')
trap = integr_g_x_1(g_x_1, -10, 10, 200, 'trapezoid')
simp = integr_g_x_1(g_x_1, -10, 10, 200, 'Simpsons')
print("Midpoint's rule:", mid, '\n Absolute error:', abs(mid - 4373 - 1/3))
print("\nTrapezoid's rule:", trap, '\n Absolute error:', abs(trap - 4373 - 1/3))
print("\nSimpson's rule:", simp, '\n Absolute error:', abs(simp - 4373 - 1/3))

Midpoint's rule: 4372.991172499997 
 Absolute error: 0.3421608333358866

Trapezoid's rule: 4374.01766 
 Absolute error: 0.6843266666670995

Simpson's rule: 4373.333334999998 
 Absolute error: 1.666665108757126e-06


**Exercise 2.2.** Write a Python function that makes a Newton-Cotes discrete approximation of the distribution of the normally distributed variable $Z \sim N(\mu,\sigma)$. Let this function take as arguments the mean $\mu$, the standard deviation $\sigma$, the number of equally spaced nodes $N$ to estimate the distribution, and the number of standard deviations $k$ away from $\mu$ to make the furthest nodes on either side of $\mu$. Use the [`scipy.stats.norm.cdf`](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy.stats.norm) command for the cdf of the normal distribution to compute the weights $\omega_n$ for the nodes $x_n$. Have this function return a vector of nodes of $[Z_1,Z_2,...Z_N]$ and a vector of weights $[\omega_1,\omega_2,...\omega_N]$ such that $\omega_i$ is given by the integral under the normal distribution between the midpoints of the two closest nodes. Define $f(Z;\mu,\sigma)$ as the pdf of the normal distribution and $F(Z;\mu,\sigma)$ as the cdf.
\begin{equation*}
  \begin{split}
    &\omega_i =
      \begin{cases}
        F\left(\frac{Z_1 + Z_2}{2};\mu,\sigma\right) \quad\quad\quad\quad\:\text{if}\quad i = 1 \\
        \int_{Z_{min}}^{Z_{max}}f(Z;\mu,\sigma)dZ \quad\quad\:\:\text{if}\quad 1<i<N \\
        1 - F\left(\frac{Z_{N-1} + Z_{N}}{2};\mu,\sigma\right) \quad\text{if}\quad i = N
      \end{cases} \\
    &\text{where}\quad Z_{min} = \frac{Z_{i-1} + Z_{i}}{2} \quad\text{and}\quad Z_{max} = \frac{Z_{i} + Z_{i+1}}{2}
  \end{split}
\end{equation*}
What are the weights and nodes $\{\omega_n,Z_n\}_{n=1}^N$ for $N=11$?

In [9]:
import scipy.stats as sts
import pandas as pd
def approx_normal(mean, sd, k, N):
    a = mean - sd * k
    b = mean + sd * k
    v_nodes = np.linspace(a, b, N)
    mids = (v_nodes[1:] + v_nodes[:-1]) / 2
    w1 = sts.norm.cdf(mids[0], mean, sd)
    wi = sts.norm.cdf(mids, mean, sd)
    wi = np.diff(wi).tolist()
    wn = 1 - sts.norm.cdf(mids[-1], mean, sd)
    v_weights = [w1] + wi + [wn]
    v_nodes = v_nodes.tolist()
    data = {'Nodes' : v_nodes, 'Weights' : v_weights}
    df = pd.DataFrame(data)
    return df

In [10]:
approx_normal(0, 1, 3, 11)

,Nodes,Weights
0,-3.0,0.003467
1,-2.4,0.014397
2,-1.8,0.048943
3,-1.2,0.117253
4,-0.6,0.198028
5,0.0,0.235823
6,0.6,0.198028
7,1.2,0.117253
8,1.8,0.048943
9,2.4,0.014397


**Exercise 2.3.** If $Z\sim N(\mu,\sigma)$, then $A\equiv e^Z\sim LN(\mu,\sigma)$ is distributed lognormally and $\log(A)\sim N(\mu,\sigma)$. Use your knowledge that $A\equiv e^Z$, $\log(A)\sim N(\mu,\sigma)$, and your function from Exercise 2.2 to write a function that gives a discrete approximation to the lognormal distribution. Note: You will not end up with evenly spaced nodes $[A_1,A_2,...A_N]$, but your weights should be the same as in Exercise 2.2.

In [11]:
def approx_lognorm(mean, sd, k, N):
    a = mean - sd * k
    b = mean + sd * k
    v_nodes = np.linspace(a, b, N)
    mids = (v_nodes[1:] + v_nodes[:-1]) / 2
    w1 = sts.norm.cdf(mids[0], mean, sd)
    wi = sts.norm.cdf(mids, mean, sd)
    wi = np.diff(wi).tolist()
    wn = 1 - sts.norm.cdf(mids[-1], mean, sd)
    v_weights = [w1] + wi + [wn]
    v_nodes = np.exp(v_nodes).tolist()
    data = {'Nodes' : v_nodes, 'Weights' : v_weights}
    df = pd.DataFrame(data)    
    return df

In [12]:
approx_lognorm(0, 1, 3, 11)

,Nodes,Weights
0,0.049787,0.003467
1,0.090718,0.014397
2,0.165299,0.048943
3,0.301194,0.117253
4,0.548812,0.198028
5,1.000000,0.235823
6,1.822119,0.198028
7,3.320117,0.117253
8,6.049647,0.048943
9,11.023176,0.014397


**Exercise 2.4.** Let $Y_i$ represent the income of individual $i$ in the United States for all individuals $i$. Assume that income $Y_i$ is lognormally distributed in the U.S. according to $Y_i\sim LN(\mu,\sigma)$, where the mean of log income is $\mu = 10.5$ and the standard deviation of log income is $\sigma = 0.8$. Use your function from Exercise 2.3 to compute an approximation of the expected value of income or average income in the U.S. How does your approximation compare to the exact expected value of $E[Y] = e^{\mu + \frac{\sigma^2}{2}}$?

In [14]:
df_lognorm = approx_lognorm(10.5, 0.8, 3, 20)
exp_inc = sum(df_lognorm.Nodes * df_lognorm.Weights)
abs_err = abs(exp_inc - np.exp(10.5 + (0.8**2)/2))
print('Approximated expected income:', exp_inc)
print('Absolute error:', abs_err)

Approximated expected income: 49994.02934637943
Absolute error: 17.05766214232426


**Exercise 3.1.** Approximate the integral of the function in Exercise 2.1 using Gaussian quadrature with $N=3$, $(\omega_1,\omega_2,\omega_3,x_1,x_2,x_3)$. Use the class of polynomials $h_i(x)=x^i$. How does the accuracy of your approximated integral compare to the approximations from Exercise 2.1 and the true known value of the integral?

In [15]:
from scipy import optimize
def poly_h(x):
    
    return [x[0] + x[1] + x[2] - 2,
           x[0] * x[3] + x[1] * x[4] + x[2] * x[5] - 0,
           x[0] * x[3]**2 + x[1] * x[4]**2 + x[2] * x[5]**2 - 2/3,
           x[0] * x[3]**3 + x[1] * x[4]**3 + x[2] * x[5]**3 - 0,
           x[0] * x[3]**4 + x[1] * x[4]**4 + x[2] * x[5]**4 - 2/5,
           x[0] * x[3]**5 + x[1] * x[4]**5 + x[2] * x[5]**5 - 0]

In [16]:
sol = optimize.root(poly_h, [0, 0, 0, 0, 0, 0])
x_vec = sol.x
a = -10
b = 10
c = (b - a) / 2
d = (b + a) / 2
gaussian_approx = c * (x_vec[0] * g_x_1(c * x_vec[3] + d) + x_vec[1] * g_x_1(c * x_vec[4] + d) + x_vec[2] * g_x_1(c * x_vec[5] + d))
gaussian_err = abs(gaussian_approx - 4373 - 1/3)
print('Gaussian approximation:', gaussian_approx)
print('Gaussian quadrature absolute error:', gaussian_err)
print('Gaussian has smaller absolute error than Newton-Cotes')

Gaussian approximation: 4373.333332841357
Gaussian quadrature absolute error: 4.919760006605323e-07
Gaussian has smaller absolute error than Newton-Cotes


**Exercise 3.2.** Use the Python Gaussian quadrature command [`scipy.integrate.quad`](http://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy.integrate.quad) to numerically approximate the integral from Exercise 2.1.
\begin{equation*}
  \int_{-10}^{10} g(x)dx \quad\text{where}\quad g(x)=0.1x^4 -1.5x^3 + 0.53x^2 + 2x + 1
\end{equation*}
How does the approximated integral using the [`scipy.integrate.quad`](http://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy.integrate.quad) command compare to the exact value of the function?

In [18]:
from scipy import integrate
quad_approx = integrate.quad(g_x_1, -10, 10)
print('Approximation:', quad_approx[0])
print('Absolute error:', abs(quad_approx[0] - 4373 - 1/3))

Approximation: 4373.333333333334
Absolute error: 6.063483048990292e-13


**Exercise 4.1.** Use Monte Carlo integration to approximate the value of $\pi$. Define a function in that takes as arguments a function $g(\mathbf{x})$ of a vector of variables $\mathbf{x}$, the domain $\Omega$ of $\mathbf{x}$, and the number of random draws $N$ and returns the Monte Carlo approximation of the integral $\int_\Omega g(\mathbf{x}) d\mathbf{x}$. Let $\Omega$ be a generalized rectangle--width $x$ and height $y$. In order to approximate $\pi$, let the functional form of the anonymous function be $g(x,y)$ from Section 4.1 with domain $\Omega = [-1,1]\times[-1,1]$. What is the smallest number of random draws $N$ from $\Omega$ that matches the true value of $\pi$ to the 4th decimal 3.1415? Set the random seed in your uniform random number generator to 25. This will make the correct answer consistent across submissions.

In [23]:
def inside_circle(x, y):
    if x ** 2 + y ** 2 <= 1:
        return 1
    else:
        return 0

In [24]:
np.random.seed = 25
N = 1
while N > 0:
    mc_draws_x = np.random.uniform(-1, 1, N)
    mc_draws_y = np.random.uniform(-1, 1, N)
    fraction = list(map(inside_circle, mc_draws_x, mc_draws_y))
    circle_area = 4 * np.mean(fraction)
    if round(circle_area, 4) == 3.1415:
        print('The smallest number of random draws:', N)
        print('Circle area:', circle_area)
        break
    else:
        N += 1

The smallest number of random draws: 834
Circle area: 3.1414868105515588


**Exercise 4.2.** Define a function in that returns the $n$-th element of a $d$-dimensional equidistributed sequence.  It should have support for the four sequences in the Table in Section 4.2.

In [27]:
def isPrime(n):
    for i in range(2, int(np.sqrt(n) + 1)):
        if n % i == 0:
            return False

    return True

def primes_ascend(N, min_val=2):
    
    primes_vec = np.zeros(N, dtype=int)
    MinIsEven = 1 - min_val % 2
    MinIsGrtrThn2 = min_val > 2
    curr_prime_ind = 0
    if not MinIsGrtrThn2:
        i = 2
        curr_prime_ind += 1
        primes_vec[0] = i
    i = min(3, min_val + (MinIsEven * 1))
    while curr_prime_ind < N:
        if isPrime(i):
            curr_prime_ind += 1
            primes_vec[curr_prime_ind - 1] = i
        i += 2

    return primes_vec

In [28]:
def equidistr_nth(n, d, seq_type):
    prime_vec = primes_ascend(d)
    if seq_type == 'Weyl':
        seq = np.sqrt(prime_vec) * n
        
    elif seq_type == 'Haber':
        seq = np.sqrt(prime_vec) * (n * (n + 1) / 2)
    
    elif seq_type == 'Niederreiter':
        seq = [n * 2 ** (i / (n + 1)) for i in range(1, d+1)]
        seq = np.array(seq)
    
    elif seq_type == 'Baker':
        seq = n * np.exp(prime_vec)
    
    seq_n = seq - np.floor(seq)
    return seq_n

In [29]:
# test
print('The 5th element of a 3-dimentional Weyl sequence', equidistr_nth(5, 3, 'Weyl'))
print('The 5th element of a 3-dimentional Haber sequence', equidistr_nth(5, 3, 'Haber'))
print('The 5th element of a 3-dimentional Niederreiter sequence', equidistr_nth(5, 3, 'Niederreiter'))
print('The 5th element of a 3-dimentional Baker sequence', equidistr_nth(5, 3, 'Baker'))

The 5th element of a 3-dimentional Weyl sequence [0.07106781 0.66025404 0.18033989]
The 5th element of a 3-dimentional Haber sequence [0.21320344 0.98076211 0.54101966]
The 5th element of a 3-dimentional Niederreiter sequence [0.61231024 0.29960525 0.07106781]
The 5th element of a 3-dimentional Baker sequence [0.94528049 0.42768462 0.06579551]


**Exercise 4.3** Repeat Exercise 4.1 to approximate the value of $\pi$, this time using quasi-Monte Carlo integration.  You will need to appropriately scale the equidistributed sequences. Compare the rates of convergence. What is the smallest number of random draws $N$ from $\Omega$ for the quasi-Monte Carlo integration that matches the true value of $\pi$ to the 4th decimal 3.1415?. Set the seed in your uniform random number generator to 25. This will make the correct answer consistent across submissions.

In [36]:
def get_smallest_draw(seq_type):
    np.random.seed = 25
    N = 1
    while N >= 0:
        x_draws = [2 * equidistr_nth(i, 2, seq_type)[0] - 1 for i in range(N)]
        y_draws = [2 * equidistr_nth(i, 2, seq_type)[1] - 1 for i in range(N)]
        fraction = list(map(inside_circle, x_draws, y_draws))
        circle_area = 4 * np.mean(fraction)
        if round(circle_area, 4) == 3.1415:
            print('Smallest number of draws for %s : %d' % (seq_type, N))
            print('Circle area:', circle_area)
            break
        elif N > 5000:
            print('%s not converging to 3.1415' % seq_type)
            break
        else:
            N += 1

In [31]:
get_smallest_draw('Weyl')

Smallest number of draws for Weyl : 1230
Circle area: 3.1414634146341465


In [32]:
get_smallest_draw('Haber')

Smallest number of draws for Haber : 2064
Circle area: 3.141472868217054


In [37]:
get_smallest_draw('Niederreiter')

Niederreiter not converging to 3.1415


In [33]:
get_smallest_draw('Baker')

Smallest number of draws for Baker : 1272
Circle area: 3.141509433962264
